# Introduction

Implementation of the representative scenario algorithm proposed in ``Bengio, Y., Frejinger, E., Lodi, A., Patel, R., & Sankaranarayanan, S. (2020, September). A learning-based algorithm to quickly compute good primal solutions for Stochastic Integer Programs. In International Conference on Integration of Constraint Programming, Artificial Intelligence, and Operations Research (pp. 99-111). Springer, Cham.``

This implementation has been developed by Warley Almeida Silva as part of the final project of the IFT 6512 Stochastic Programming course. The course has been taught by professor Fabian Bastin at Université de Montréal during the Winter semester of 2022. The problem considered in this implementation is the Stochastic Capacitated Facility Location Problem (SCFLP). Please find the latest version of the report [here](https://github.com/almeidawarley/ift6512).

Please go straight to the last section "Procedures" to see the overview of the steps conducted to write the report.

# Packages

``JuMP`` has been used to model mixed-integer programs, whereas ``Gurobi`` has been used to solve mixed-integer programs. ``Suppresssor`` has been used to suppress the output of the solver. ``MultivariateStats`` has been used to train the linear regression model with Ridge regression. ``Random``, ``Statistics``, ``Distributions``, and ``StatsBase`` have been used to generate instances and perform some statistical operations.

In [1]:
using JuMP, Gurobi
using Suppressor
using MultivariateStats
using Random, Statistics
import Distributions
using StatsBase

# Auxiliaries

Function ``stringify_vector`` transforms a vector into a string.

In [2]:
function stringify_vector(v, r)
    lines = [string(round(v[i], digits=6)) for i = 1:r]
    return join(lines, ",")
end

stringify_vector (generic function with 1 method)

Function ``stringify_matrix`` transforms a matrix into a string.

In [3]:
function stringify_matrix(v, r, c)
    lines = [stringify_vector(v[i,:], c) for i = 1:r]
    return join(lines, "\n")
end

stringify_matrix (generic function with 1 method)

# Programs

Function ``deterministic_SCFLP`` builds program $SCFLP^D$.

In [4]:
function deterministic_SCFLP(n, s, cf, cv, ctf, ctv, d, M)
    # Build the deterministic SCFLP model    
    
    # Create sets L, H and Ξ
    L = 1:n # Set of locations
    H = 1:(n+1) # Locations with hub
    Ξ = 1:s # Set of scenarios
    
    # Create model object
    model = Model(Gurobi.Optimizer)
    
    # Create 1st stage variables
    @variable(model, b[L], binary = true)
    @variable(model, v[L], lower_bound = 0)
    
    # Create 2nd stage variables
    @variable(model, u[H, L, Ξ], binary = true)
    @variable(model, y[H, L, Ξ], lower_bound = 0)
    
    # Create 1st stage constraints
    @constraint(model, c1, 0.10*n <= sum(b) <= 0.75*n)
    @constraint(model, c2[i = L], v[i] <= M * b[i])
    
    # Create 2nd stage constraints
    @constraint(model, c3[i = L, s = Ξ], sum(y[i, :, s]) <= v[i])
    @constraint(model, c4[j = L, s = Ξ], sum(y[:, j, s]) == d[j, s])
    @constraint(model, c5[i = H, j = L, s = Ξ], y[i, j, s] <= M * u[i, j, s])
    
    # Create objective function
    @objective(model, Min, 
        # 1st stage objective
        cf' * b + cv' * v
        # 2nd stage objective
        + (1/s) * sum(ctf[i, j] * u[i, j, s] + ctv[i, j] * y[i, j, s] for i = H, j = L, s = Ξ)
    )
    
    # println("Deterministic SCFLP model built successfully")
    return model, b, v, u, y
end

deterministic_SCFLP (generic function with 1 method)

Function ``surrogate_SCFLP`` builds program $SCFLP^S$.

In [5]:
function surrogate_SCFLP(n, s, cf, cv, ctf, ctv, d, M)
    # Build the surrogate SCFLP model
    
    # Create sets L and H
    L = 1:n # Set of locations
    H = 1:(n+1) # Locations with hub
    
    # Create model object
    model = Model(Gurobi.Optimizer)
    
    # Create 1st stage variables
    @variable(model, b[L], binary = true)
    @variable(model, v[L], lower_bound = 0)
    
    # Create 2nd stage variables
    @variable(model, u[H, L], binary = true)
    @variable(model, y[H, L], lower_bound = 0)
    
    # Create 1st stage constraints
    @constraint(model, c1, 0.10*n <= sum(b) <= 0.75*n)
    @constraint(model, c2[i = L], v[i] <= M * b[i])
    
    # Create 2nd stage constraints
    @constraint(model, c3[i = L], sum(y[i, :]) <= v[i])
    @constraint(model, c4[j = L], sum(y[:, j]) == d[j])
    @constraint(model, c5[i = H, j = L], y[i, j] <= M * u[i,j])
    
    # Create objective function
    @objective(model, Min, 
        # 1st stage objective
        cf' * b + cv' * v
        # 2nd stage objective
        + sum(ctf[i, j] * u[i, j] + ctv[i, j] * y[i, j] for i = H, j = L)
    )
    
    # println("Surrogate SCFLP model built successfully")
    return model, b, v, u, y
end

surrogate_SCFLP (generic function with 1 method)

Function ``restricted_SCFLP`` builds program $SCFLP^D$ with a fixed first-stage decision $(b,v)$.

In [6]:
function restricted_SCFLP(n, s, cf, cv, ctf, ctv, d, M, b, v)
    # Build the restricted SCFLP model for (b,v)
    
    # Create sets L, H and Ξ
    L = 1:n # Set of locations
    H = 1:(n+1) # Locations with hub
    Ξ = 1:s # Set of scenarios
    
    # Create model object
    model = Model(Gurobi.Optimizer)
    
    # Create 2nd stage variables
    @variable(model, u[H, L, Ξ], binary = true)
    @variable(model, y[H, L, Ξ], lower_bound = 0)
    
    # Create 2nd stage constraints
    @constraint(model, c3[i = L, s = Ξ], sum(y[i, :, s]) <= v[i])
    @constraint(model, c4[j = L, s = Ξ], sum(y[:, j, s]) == d[j, s])
    @constraint(model, c5[i = H, j = L, s = Ξ], y[i, j, s] <= M * u[i, j, s])
    
    # Create objective function
    @objective(model, Min, 
        # 1st stage objective
        cf' * b + cv' * v
        # 2nd stage objective
        + (1/s) * sum(ctf[i, j] * u[i, j, s] + ctv[i, j] * y[i, j, s] for i = H, j = L, s = Ξ)
    )
    
    # println("Restricted SCFLP model built successfully")
    return model, b, v, u, y
end

restricted_SCFLP (generic function with 1 method)

# Instances

Function ``instance_standard`` returns standard instance information.

In [7]:
function instance_standard()
    # Standard parameters
    
    n = 10 # Number of locations
    s = 50 # Number of realizations
    # Normal transportation costs
    ctf = zeros(n + 1, n)
    ctv = zeros(n + 1, n)
    for i = 1:n
        for j = 1:n
            ctf[i, j] = 10 * abs(i-j)
            ctv[i, j] = abs(i-j)
        end
    end
    # Hub transportation costs
    for j = 1:n
        ctf[n + 1, j] = 5 * maximum(ctf[:, j])
        ctv[n + 1, j] = 5 * maximum(ctv[:, j])
    end
    return n, s, ctf, ctv
end

instance_standard (generic function with 1 method)

Function ``instance_variable`` returns variable instance information.

In [8]:
function instance_variable(n, s)
    # Variable parameters
    
    # Installation costs
    cf = rand(Distributions.Uniform(15,20), n)
    cv = rand(Distributions.Uniform(5,10), n)
    # Demand values
    λ = floor.((cf + 10 * cv)/sqrt(n))
    d = zeros(n, s)
    for i = 1:n
        d[i, :] = rand(Distributions.Poisson(λ[i]), s)
    end
    # Big M constant
    M = 1000
    return cf, cv, d, M
end

instance_variable (generic function with 1 method)

Function ``instance_generator`` creates and stores instances.

In [9]:
function instance_generator(instances)
    # Create and store instances
    
    # Retrieve standard instance information
    n, s, ctf, ctv = instance_standard()
    for counter = instances
        # Retrieve variable instance information
        cf, cv, d, M = instance_variable(n, s)
        # Write instance file in the instance folder
        filename = "instances/instance-"*string(counter)*".txt"
        open(filename, "w") do io
            line = string(n) * "," * string(s) * "," * string(M) * "\n"
            write(io, line)
            write(io, stringify_vector(cf, n) * "\n")
            write(io, stringify_vector(cv, n) * "\n")
            write(io, stringify_matrix(ctf, n+1, n) * "\n")
            write(io, stringify_matrix(ctv, n+1, n) * "\n")
            write(io, stringify_matrix(d, n, s) * "\n")           
        end
    end
end

instance_generator (generic function with 1 method)

Function ``instance_reader`` reads an instance from file.

In [10]:
function instance_reader(identifier)
    # Read instance file from the instance folder
    
    filename = "instances/instance-" * string(identifier) * ".txt"
    open(filename, "r") do io
        n, s, M = parse.(Int64, split(readline(io), ","))
        cf = parse.(Float64, split(readline(io), ","))
        cv = parse.(Float64, split(readline(io), ","))
        ctf = zeros(n + 1, n)
        for i = 1:(n+1)
            ctf[i, :] = parse.(Float64, split(readline(io), ","))
        end
        ctv = zeros(n + 1, n)
        for i = 1:(n+1)
            ctv[i, :] = parse.(Float64, split(readline(io), ","))
        end
        d = zeros(n, s)
        for i = 1:n
            d[i, :] = parse.(Float64, split(readline(io), ","))
        end
        return n, s, cf, cv, ctf, ctv, d, M
    end
end

instance_reader (generic function with 1 method)

Function ``output_exporter`` exports solver output to file.

In [11]:
function output_exporter(identifier, model, b, v)
    # Export solver output of the deterministic SCFLP
    
    objective = string(round(objective_value(model), digits=6))
    status = string(termination_status(model))
    gap = string(round(relative_gap(model), digits=6))
    time = string(round(solve_time(model), digits=6))
    # Write solver output to the output folder
    filename = "outputs/output-"*string(identifier)*".out"
    open(filename, "w") do io
        line = objective * "," * status * "," * gap
        write(io, line * "\n")
        write(io, stringify_vector(value.(b), length(value.(b))) * "\n")
        write(io, stringify_vector(value.(v), length(value.(v))) * "\n")
    end
end

output_exporter (generic function with 1 method)

Function ``output_reader`` reads a solver output file.

In [12]:
function output_reader(identifier)
    # Read output file of the determinisitc SCFLP from the output folder
    
    filename = "outputs/output-" * string(identifier) * ".out"
    open(filename, "r") do io
        objective, status, gap = split(readline(io), ",")
        objective = parse(Float64, objective) 
        gap = parse(Float64, gap)
        b = parse.(Float64, split(readline(io), ","))
        v = parse.(Float64, split(readline(io), ","))
        n, s, cf, cv, ctf, ctv, d, M = instance_reader(identifier)
        return n, s, cf, cv, ctf, ctv, d, M, objective, status, gap, b, v
    end
end

output_reader (generic function with 1 method)

# Deterministic

Function ``run_deterministic`` solves $SCFLP^D$ for some instances.

In [13]:
function run_deterministic(instances)
    # Run deterministic SCFLP for some instances
    
    for identifier = instances
        println("Reading file for instance #", identifier)
        n, s, cf, cv, ctf, ctv, d, M = instance_reader(identifier)
        println("Building deterministic model for instance #", identifier)
        deterministic, b, v, u, y = deterministic_SCFLP(n, s, cf, cv, ctf, ctv, d, M)
        set_time_limit_sec(deterministic, 10 * 60)
        set_optimizer_attribute(deterministic, "MIPGap", 0.02)
        println("Optimizing deterministic model for instance #", identifier)
        @suppress begin
            optimize!(deterministic)
        end
        println("Exporting result for instance #", identifier)
        output_exporter(identifier, deterministic, b, v)
    end
end

run_deterministic (generic function with 1 method)

# Heuristic

Function ``representative_scenario_finder`` runs the heuristic for an instance.

In [14]:
function representative_scenario_finder(identifier, iterations = 100, c = 0.01)
    # Find a representative scenario
    
    # Read the deterministic SCFLP solution
    n, s, cf, cv, ctf, ctv, d, M, objective, status, gap, b_opt, v_opt = output_reader(identifier)
    # Create an average realization
    d_bar = mean(d[:,i] for i = 1:s)
    
    counter = 0
    while counter <= iterations
        # Solve the surrogate SCFLP for the current realization
        surrogate, b, v, u, y = surrogate_SCFLP(n, s, cf, cv, ctf, ctv, d_bar, M)
        @suppress begin
            optimize!(surrogate)
        end
        
        # Retrieve information about the surrogate SCFLP solution
        objective_bar = objective_value(surrogate)
        b_bar = value.(b)
        v_bar = value.(v)
        
        # Calculate the ratio between the deterministic and current surrogate solution        
        ratio = objective_bar/objective
        if ratio >= 1 - c && ratio <= 1 + c 
            # The surrogate objective value is sufficiently close to the optimal
            println("Accepted a representative scenario for instance #", identifier, " at iteration ", counter)
            return true, d_bar
        else
            # The surrogate objective value is not sufficiently close to the optimal
            # Apply heuristics to perturbate the current realization
            if counter == 0
                d_bar = first_perturbation(d_bar, b_opt, b_bar, v_opt, v_bar)
            else
                d_bar = additional_perturbation(d_bar, b_opt, b_bar, v_opt, v_bar)
            end
        end
        counter += 1
    end
    return false, d_bar
end

representative_scenario_finder (generic function with 3 methods)

Function ``first_perturbation`` runs a simple first perturbation.

In [15]:
function first_perturbation(d_bar, b_opt, b_bar, v_opt, v_bar)
    # First type of perturbation
    
    for i = 1:length(b_opt)
       if b_opt[i] == 0 && b_bar[i] == 1
           d_bar[i] = 0
       end
    end
    return d_bar
end

first_perturbation (generic function with 1 method)

Function ``additional_perturbation`` runs additional perturbations.

In [16]:
function additional_perturbation(d_bar, b_opt, b_bar, v_opt, v_bar, p = 0.5)
    # Second type of perturbation
    
    k = 1
    for i = 2:length(b_opt)
       if abs(v_opt[i] - v_bar[i]) >= abs(v_opt[k] - v_bar[k])
           k = i    
       end
    end
    sign = (v_opt[k] - v_bar[k])/abs(v_opt[k] - v_bar[k])
    d_bar[k] = d_bar[k] + sign * p * d_bar[k]   
    return d_bar
end

additional_perturbation (generic function with 2 methods)

Function ``run_heuristic`` runs the heuristic for some instances.

In [17]:
function run_heuristic(instances)
    # Run the heuristic for some instances
    
    for identifier = instances
        # Call the heuristic with a maximum of 1000 iterations
        flag, d = representative_scenario_finder(identifier, 1000)
        if flag
            # Write representative scenario in the representative scenario folder
            filename = "scenarios/scenario-" * string(identifier) * ".sec"
            open(filename, "w") do io
                write(io, stringify_vector(d, length(d)) * "\n")       
            end
        else
            # Report that the heuristic was not successful for some instance
            println("Missing representative scenario for instance #", identifier)
        end
    end
end

run_heuristic (generic function with 1 method)

# Linear regression

Function ``instance_features`` molds the input to the linear regression model.

In [18]:
function instance_features(identifier)
    # Mold the input to the linear regression model
    
    # Read instance information    
    n, s, cf, cv, ctf, ctv, d, M = instance_reader(identifier)
    
    # General features
    d_min = minimum.(d[i,:] for i = 1:n)
    d_max = maximum.(d[i,:] for i = 1:n)
    d_avg = mean.(d[i, :] for i = 1:n)
    d_std = std.(d[i, :] for i = 1:n)
    d_med = median.(d[i, :] for i = 1:n)
    
    # Quantile features
    qntl = quantile.(d[i, :] for i = 1:n)
    d_q1 = zeros(n)
    d_q3 = zeros(n)
    for i = 1:n
        d_q1[i] = qntl[i][2]
        d_q3[i] = qntl[i][4]
    end
    
    # Additional features
    sml = zeros(5, n)
    grt = zeros(5, n)    
    index = 1
    cs = [0.9 1.0 1.1 1.2 1.5]
    for c = cs
        for i = 1:n
            smaller_counter = 0
            greater_counter = 0
            for k = 1:s
                smaller_flag = true
                greater_flag = true
                for j = 1:n
                    if c * d[i, k] < d[j, k]
                        greater_flag = false
                    end
                    if c * d[i, k] > d[j, k]
                        smaller_flag = false
                    end
                end
                if smaller_flag
                    smaller_counter += 1
                end
                if greater_flag
                    greater_counter += 1
                end
            end
            sml[index, i] = smaller_counter/s
            grt[index, i] = greater_counter/s
        end
        index += 1
    end  
    
    # Reshape procedure to return a vector with 190 lines
    sml = reshape(sml, (:,1))
    grt = reshape(grt, (:,1))
    return [cf; cv; d_min; d_max; d_avg; d_std; d_med; d_q1; d_q3; sml; grt]
end

instance_features (generic function with 1 method)

Function ``representate_scenario_reader`` reads the representative scenario from a file.

In [19]:
function representative_scenario_reader(identifier)
    # Read representative scenario from the representative scenario folder
    
    filename = "scenarios/scenario-" * string(identifier) * ".sec"
    open(filename, "r") do io
        d = parse.(Float64, split(readline(io), ","))
        return d
    end
end

representative_scenario_reader (generic function with 1 method)

Function ``train_regression_model`` trains the linear regression model.

In [20]:
function train_regression_model(instances)
    # Train the linear regression model
    
    # Retrieve input-output pairs for the training set
    println("Fetching information from files")
    inputs = [instance_features(identifier) for identifier = instances]
    labels = [representative_scenario_reader(identifier) for identifier = instances]    
    # Create auxiliary variables
    number = length(inputs)
    attributes = length(inputs[1])
    scenarios = length(labels[1])
    # Create input and label matrices
    println("Generating training matrices")
    trainX = zeros(number, attributes)
    trainY = zeros(number, scenarios)
    for i = 1:number
        trainX[i, :] = inputs[i]
        trainY[i, :] = labels[i]
    end
    println("Training the linear regression model")
    # Call the Ridge Regression function
    solution = ridge(trainX, trainY, 0.00001)
    A, b = solution[1:end-1,:], solution[end,:]
    return A, b
end

train_regression_model (generic function with 1 method)

Function ``create_regression_model`` trains and stores the linear regression model.

In [21]:
function create_regression_model(size)
    # Create the linear regression model
    
    # Retrieve the set of trainable instances
    trainable = retrieve_set_lambda()
    # Sample enough instances randomly
    training = sample(trainable, size, replace = false)
    # Write sampled instances into a file
    filename = "training.txt"
    open(filename, "w") do io
        write(io, stringify_vector(training, length(training)) * "\n")       
    end
    # Train the linear regression model
    @time A, b = train_regression_model(training)
    A = A'
    # Write linear regression model into a file
    filename = "regression.txt"
    open(filename, "w") do io
        write(io, stringify_matrix(A, 10, 190) * "\n")
        write(io, stringify_vector(b, 10) * "\n")
    end
end

create_regression_model (generic function with 1 method)

# Persistence

Function ``retrieve_set_lambda`` retrieves instances with a representative scenario.

In [22]:
function retrieve_set_lambda()
    # Retrieve instances with a representative scenario
    
    # Scan the representative scenario folder
    instances = []
    for filename = readdir("scenarios/")
        if occursin(".sec", filename)
            filename = replace(filename, "scenario-" => "")
            filename = replace(filename, ".sec" => "")
            append!(instances, parse(Int64, filename))
        end
    end
    return instances
end

retrieve_set_lambda (generic function with 1 method)

Function ``retrieve_training_set`` retrieves instances in the training set.

In [23]:
function retrieve_training_set()
    # Retrieve instances in the training set
    
    filename = "training.txt"
    open(filename, "r") do io
        training = parse.(Float64, split(readline(io), ","))
        training = floor.(Int64, training)
        return training
    end
end

retrieve_training_set (generic function with 1 method)

Function ``retrieve_regression_model`` reads the linear regression model.

In [24]:
function read_regression_model()
    # Read linear regression model from file
    
    filename = "regression.txt"
    open(filename, "r") do io
        A = zeros(10, 190)
        for i = 1:10
            A[i, :] = parse.(Float64, split(readline(io), ","))
        end
        b = parse.(Float64, split(readline(io), ","))
        return A, b
    end
end

read_regression_model (generic function with 1 method)

# Approaches

Function ``AVG`` runs the average approach.

In [25]:
function AVG(identifier)
    # AVG approach explained in the report
    
    println("Running AVG approach for instance #", identifier)
    n, s, cf, cv, ctf, ctv, d, M = instance_reader(identifier)
    println("Data retrieved from instance file successfully")
    d_bar = mean(d[:,i] for i = 1:s)
    println("Average scenario as representative scenario generated successfully")
    surrogate, b, v, u, y = surrogate_SCFLP(n, s, cf, cv, ctf, ctv, d_bar, M)
    println("Surrogate model acquired successfully")
    set_time_limit_sec(surrogate, 10 * 60)
    optimize!(surrogate)
    return surrogate, b, v
end

AVG (generic function with 1 method)

Function ``RND`` runs the random approach.

In [26]:
function RND(identifier)
    # RND approach explained in the report
    
    println("Running RND approach for instance #", identifier)
    n, s, cf, cv, ctf, ctv, d, M = instance_reader(identifier)
    println("Data retrieved from instance file successfully")
    d_bar = d[:, rand(1:s)]
    println("Random scenario as representative scenario generated successfully")
    surrogate, b, v, u, y = surrogate_SCFLP(n, s, cf, cv, ctf, ctv, d_bar, M)
    println("Surrogate model acquired successfully")
    set_time_limit_sec(surrogate, 10 * 60)
    optimize!(surrogate)
    return surrogate, b, v
end

RND (generic function with 1 method)

Function ``LR`` runs the linear regression approach.

In [27]:
function LR(identifier)    
    # LR approach explained in the report
    
    println("Running LR approach for instance #", identifier)
    n, s, cf, cv, ctf, ctv, d, M = instance_reader(identifier)
    println("Data retrieved from instance file successfully")
    fts = instance_features(identifier)
    A, b = read_regression_model()
    d_bar = A * fts + b
    println("Linear regression scenario as representative scenario generated successfully")
    surrogate, b, v, u, y = surrogate_SCFLP(n, s, cf, cv, ctf, ctv, d_bar, M)
    println("Surrogate model acquired successfully")
    set_time_limit_sec(surrogate, 10 * 60)
    optimize!(surrogate)
    return surrogate, b, v
end

LR (generic function with 1 method)

Function ``GRB`` runs the Gurobi approach.

In [28]:
function GRB(identifier)
    # GRB approach explained in the report
    
    println("Running GRB approach for instance #", identifier)
    n, s, cf, cv, ctf, ctv, d, M = instance_reader(identifier)
    println("Data retrieved from instance file successfully")
    deterministic, b, v, u, y = deterministic_SCFLP(n, s, cf, cv, ctf, ctv, d, M)
    println("Deterministic model acquired successfully")
    set_time_limit_sec(deterministic, 10 * 60)
    set_optimizer_attribute(deterministic, "MIPGap", 0.02)
    optimize!(deterministic)
    return deterministic, b, v
end

GRB (generic function with 1 method)

Function ``APR`` runs the approximate approach.

In [29]:
function APR(identifier, number)
    # APR(k) approach explained in the report
    
    println("Running APR(",number,") approach for instance #", identifier)
    n, s, cf, cv, ctf, ctv, d, M = instance_reader(identifier)
    println("Data retrieved from instance file successfully")
    if number < 1 || number > s
        return nothing
    end
    # Sample k realizations from the support set
    selected = sample(1:s, number, replace = false)
    d_bar = d[:, selected[1]]
    for index = 2:number
        d_bar =  hcat(d_bar, d[:, selected[index]])
    end
    println("Approximated the support set with ", number, " random scenarios successfully")
    deterministic, b, v, u, y = deterministic_SCFLP(n, number, cf, cv, ctf, ctv, d_bar, M)
    println("Deterministic model acquired successfully")
    set_time_limit_sec(deterministic, 10 * 60)
    set_optimizer_attribute(deterministic, "MIPGap", 0.02)
    optimize!(deterministic)
    return deterministic, b, v
end

APR (generic function with 1 method)

Function ``HYB`` runs the hybrid approach.

In [30]:
function HYB(identifier)
    # HYB approach explained in the report
    
    println("Running HYB approach for instance #", identifier)
    n, s, cf, cv, ctf, ctv, d, M = instance_reader(identifier)
    println("Data retrieved from instance file successfully")
    # AVG approach
    average = mean(d[:,i] for i = 1:s)
    # RND approach
    random = d[:, rand(1:s)]
    # LR approach
    fts = instance_features(identifier)
    A, b = read_regression_model()
    linear = A * fts + b
    # Ensemble support set
    number = 3
    d_bar =  hcat(average, random, linear)
    println("Ensemble built with ", number, " representative scenarios successfully")
    deterministic, b, v, u, y = deterministic_SCFLP(n, number, cf, cv, ctf, ctv, d_bar, M)
    println("Deterministic model acquired successfully")
    set_time_limit_sec(deterministic, 10 * 60)
    set_optimizer_attribute(deterministic, "MIPGap", 0.02)
    optimize!(deterministic)
    return deterministic, b, v
end

HYB (generic function with 1 method)

# Experiments

Function ``compilate_approaches`` compilates the approaches.

In [31]:
function compilate_approches()
    # Compilate approaches
    
    AVG(1)
    LR(1)
    RND(1)
    GRB(1)
    HYB(1)
    APR(1,2)
    println("Approaches compiled successfully")
end

compilate_approches (generic function with 1 method)

Function ``logger`` stores the output of the approaches.

In [32]:
function logger(identifier, model, b, v, tag, time)
    # Log the general results per instance and detailed solutions per instance and approach
    
    objective = string(round(objective_value(model), digits=6))
    status = string(termination_status(model))
    gap = string(round(relative_gap(model), digits=6))
    time = string(round(time, digits=6))
    # The result folder has one file per instance with information about all approaches
    filename = "results/results-"*string(identifier)*".txt"
    open(filename, "a") do io
        line = string(identifier) * "," * tag * "," * time * "," 
        line = line * objective * "," * status * "," * gap
        write(io, line * "\n")
    end
    # The detailed folder has one file per instance and approach with the solution
    filename = "detailed/"*tag*"-"*string(identifier)*".txt"
    open(filename, "w") do io
        line = objective * "," * status * "," * gap
        write(io, line * "\n")
        write(io, stringify_vector(value.(b), length(value.(b))) * "\n")
        write(io, stringify_vector(value.(v), length(value.(v))) * "\n")
    end
end

logger (generic function with 1 method)

Function ``run_experiments`` runs the computational experiments for some instances.

In [33]:
function run_experiments(instances)
    # Run the computational experiments of the paper    
    
    trainable = retrieve_set_lambda()
    training = retrieve_training_set()
    compilate_approaches()

    for identifier = instances
        # For the instance set Γ, use "if !(identifier in trainable)"
        # Run approaches for the testing set drawn the instance set Λ
        if identifier in trainable && !(identifier in training)
            time = @elapsed model, b, v = AVG(identifier)
            logger(identifier, model, b, v, "avg", time)
            time = @elapsed model, b, v = LR(identifier)
            logger(identifier, model, b, v, "lr", time)
            time = @elapsed model, b, v = GRB(identifier)
            logger(identifier, model, b, v, "grb", time)
            time = @elapsed model, b, v = RND(identifier)
            logger(identifier, model, b, v, "rnd", time)
            time = @elapsed model, b, v = HYB(identifier)        
            logger(identifier, model, b, v, "hyb", time)
            time = @elapsed model, b, v = APR(identifier, 5)
            logger(identifier, model, b, v, "apr5", time)
            time = @elapsed model, b, v = APR(identifier, 10)
            logger(identifier, model, b, v, "apr10", time)
            time = @elapsed model, b, v = APR(identifier, 15)
            logger(identifier, model, b, v, "apr15", time)
            time = @elapsed model, b, v = APR(identifier, 20)
            logger(identifier, model, b, v, "apr20", time)
            time = @elapsed model, b, v = APR(identifier, 25)
            logger(identifier, model, b, v, "apr25", time)
        end
    end
end

run_experiments (generic function with 1 method)

Funcion ``run_restricted`` solves $SCFLP^D$ with fixed first-stage decisions per approach.

In [34]:
function run_restricted()
    # Run the restricted SCFLP per instance and approach
    
    for filename = readdir("detailed/")
        if occursin(".txt", filename)
            arguments = replace(filename, ".txt" => "")
            arguments = split(arguments, "-")
            tag = arguments[1]
            identifier = parse(Int64, arguments[2])
            b_0 = nothing
            v_0 = nothing
            # Retrieve the first-stage decision of the approach
            open("detailed/"* filename, "r") do io
                objective, status, gap = split(readline(io), ",")
                b_0 = parse.(Float64, split(readline(io), ","))
                v_0 = parse.(Float64, split(readline(io), ","))                
            end
            # Build and solve the restricted SCFLP
            n, s, cf, cv, ctf, ctv, d, M = instance_reader(identifier)
            re, _, _, _, _ = restricted_SCFLP(n, s, cf, cv, ctf, ctv, d, M, b_0, v_0)
            optimize!(re)
            respective = objective_value(re)
            # Store the objective value of the restricted SCFLP in the constrained folder
            # The constrained folder has one file per instance with information about all approaches
            filename = "constrained/constrained-"* string(identifier) * ".txt"
            open(filename, "a") do io
                line = string(identifier) * "," * tag * "," * string(respective)
                write(io, line * "\n")
            end
            print(".")
        end
    end
end

run_restricted (generic function with 1 method)

# Procedures

In the preliminary computations, the project creates some instances, solves $SCFLP^D$ for them, and finds representatives scenarios for as much instances as possible. Lastly, the project creates the linear regression model by sampling a certain number of instances for the training set.

In [35]:
# instance_generator(1:50000)

In [36]:
# run_deterministic(1:50000)

In [37]:
# run_heuristic(1:50000)

In [38]:
# create_regression_model(45000)

In the computational experiments, the project runs the different approaches for a certain instance to (i) find a first-stage solution per approach and (ii) store the computational time. Afterwards, the project solves program $SCFLP^D$ with fixed first-stage decisions per approach to calculate the objective ratio.

In [39]:
# run_experiments(1:50000)

In [41]:
# run_restricted()

The tables and graphs presented in the report have been generated at a later stage based on the exported files.